# 03_model_25D_5fold.ipynb

This script contain the model training (using the training set) and making predictions on the independent test set. The model is trained by applying 5-fold cross validation.

Required :
- (suggest) use of GPU
- the 2.5D images in JPEG (.jpeg) format
- `local_label.csv`

Generate:
- the model structure and weight (.h5)
- prediction result on the training-validation set (`trainval_5fold_df.csv`)
- prediction result on the independent test set (`test_5fold_df.csv`)

In [1]:
import os
import pandas as pd
import numpy as np
from glob import glob
from sklearn.model_selection import GroupShuffleSplit, GroupKFold

pd.set_option("display.max_columns", 20)
pd.set_option("display.max_colwidth", 150)

import tensorflow.keras
import tensorflow as tf
from tensorflow.keras.experimental import CosineDecay
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

# utilitiy files located in the src folder
from src.util_data import check_corrupted, define_classes, find_binary_class_weights
from src.util_plot import show_one_image, show_augmentation_layers
from src.util_model import get_data_augmentation_layers_keras, get_imgaug_augmentation
from src.util_model import load_image_and_label_from_path
from src.util_model import create_model_enb1

### Check if TensorFlow is running on GPU

In [ ]:
from tensorflow.python.client import device_lib 
# print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Expected output (for running on a single GPU):
```
Num GPUs Available:  1
```

In [ ]:
# locating the folder paths used in this script.
work_dir = <PATH TO project_dir> # path to the directory of the current project (project master path)
data_folder_path = work_dir + "CT_data/" # where the data is stored
image_folder_path = data_folder_path + "CT_25DJPG_cropped_combineLR_downsampled/" # directory of the 2.5D images that generated by 02_NIFTI_to_25DJPG.ipynb 


# load the binary label and add the full file path of the 2.5D JPEG files
df_label = pd.read_csv(image_folder_path + "local_label.csv")
df_label["filepath"] = image_folder_path
df_label["filepath"] = df_label["filepath"].str.cat(df_label["jpeg_name"],sep="")

display(df_label)

In [4]:
# Double check if there are any corrupted file path
drop_i_row = check_corrupted(df_label,"filepath")
df_label = df_label.drop(drop_i_row) 

### Display samples
1 normal adrenal, and 1 abnormal adrenal.

In [ ]:
# Examples of whole lung vs others. Coronal slice in current path.
print("Abnormal adrenal:")
image_path = df_label.filepath[df_label.abnormal==1].iloc[10] # showing the 10th image (w.r.t the DataFrame df_label) that has the abnormal flag = 1 (abnormal example).
show_one_image(image_path)

print("Normal adrenal:")
image_path = df_label.filepath[df_label.abnormal==0].iloc[10] # showing the 10th image (w.r.t the DataFrame df_label) that has the abnormal flag = 0 (normal example).
show_one_image(image_path)

### Input the parameters for the model training

In [6]:
######################## INPUT VALUES ##########################
# image data related parameters
batch_size = 64 # number of images contained in one batch
image_size_h = 240 # input image size (height)
image_size_w = 120 # input image size (width)

# image augmentation related parameter
aug_rotation = (-0.05,0.05) # random rotation
aug_zoom = (-0.2, 0) # random zoom
aug_contrast = (0.2,0.2) # random contrast
aug_coarsedropout = (0.0, 0.10) # random coarse dropout
aug_invert = 0.5 # random invert

# path and name for the model weight to save 
set_model_tag = "ENB1" # ENB1 = EfficientNet version B1
set_model_path = <CURRENT DIRECTORY> + "/model_h5/" # set the folder for the model and its weight (.h5) to save

# model training related parameters
epochs = 8 # number of epochs
dropout_rate = 0.2 # droupout rate of the model
initial_learning_rate = 1e-4 # initial learning rate
alpha = 0.3 # alpha value
################################################################

input_shape = (image_size_h, image_size_w, 3)

In [ ]:
# Get number of images and patient from the patient list
patient_list = df_label['PatientName'].tolist()
print('Number of 2.5D images = ', len(patient_list))
patient_list = list(dict.fromkeys(patient_list))
print('Number of patient = ', len(patient_list))

### Spliting the dataset (training and testing)
Dataset split into training set and independent test set with a ratio of $80\%:20\%$. The split is in unit of patients. Any patient can only be allocated in rather training or test set (never appear in both set). The training set is used for the 5-fold cross validation.

In [ ]:
# data split (train and test set)
test_size=0.2 # set the test set ratio

# split data into training set (train_val_df) and test set (test_df).
gss = GroupShuffleSplit(n_splits=2, test_size=test_size, random_state=5)
train_val_id, test_id = next(gss.split(df_label, groups=df_label['PatientName']))
train_val_df = df_label.iloc[train_val_id]
test_df = df_label.iloc[test_id]

# print the number count in each set
print('Number of 2.5D images in the train-val set: ',len(train_val_df))
print('Number of 3D images in the train-val set:   ',len(train_val_df.image_name.value_counts()))
print('Number of patients in the train-val set:    ', len(train_val_df.PatientName.value_counts()))
print('--------------------------------------------------')
print('Number of 2.5D images in the test set:      ',len(test_df))
print('Number of 3D images in the test set:        ',len(test_df.image_name.value_counts()))
print('Number of patients in the test set:         ', len(test_df.PatientName.value_counts()))
print('--------------------------------------------------')

# double check if there's any patient is allocated in both the train and test set.
set_A = set(train_val_id)
set_B = set(test_id)
# 'Same element!' if there is any element is the same
# 'No same element.' if there is no element is the same
output = 'Good! No same element.' if (set_A.intersection(set_B) == set()) else 'Same element! Please double check.'
print(output)
#if True (there is any element same), show which one are in both sets.
print([i for i in set_B if i in set_A])

Expected output:
```
Number of 2.5D images in the train-val set:  x
Number of 3D images in the train-val set:    y
Number of patients in the train-val set:     z
--------------------------------------------------
Number of 2.5D images in the test set:       X
Number of 3D images in the test set:         Y
Number of patients in the test set:          Z
--------------------------------------------------
Good! No same element.
[]
```
Note that the statement `Good! No same element` is important to make sure that there no single patient allocated into both training and test set. There are risks of data leaking (to/from the test set) if the above statement is not printed. If there are any samples allocated into both training and test set, they will be listed (if the list is empty: `[]`, which means no samples are allocated repeatedly).

### Image augmentation

In [9]:
# Set up the image augmentation layers
data_augmentation_layers = get_data_augmentation_layers_keras(aug_rotation=aug_rotation, aug_zoom=aug_zoom, aug_contrast=aug_contrast)
seq = get_imgaug_augmentation(aug_coarsedropout, aug_invert)

In [10]:
def augment_batch(image, label):
    """
    Function to apply the augmentation to the batch.
    """
    def augment_image(image):
        return seq.augment(image=image.numpy())
    image = tf.cast(image, tf.uint8)
    image = tf.py_function(augment_image, [image], tf.uint8)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE

Demonstrate the effect of the augmentation layers (keras).

In [ ]:
image_path = df_label.filepath.iloc[46]
show_augmentation_layers(image_path,data_augmentation_layers)

### Preparing the model and training

In [12]:
# define the classes and get number of classes
classes_to_predict, num_classes = define_classes(train_val_df.abnormal)

# set the K-Fold for the training set
k_fold = 5 # number of fold to train. 5 = 5-fold.
gkf = GroupKFold(n_splits=k_fold)

In [ ]:
# View model structure (using EfficientNet B1)
model_efficientnetb1 = create_model_enb1(input_shape=input_shape,
                                         num_classes=num_classes,
                                         dropout_rate=dropout_rate,
                                         data_augmentation_layers=data_augmentation_layers)
#print the model structure summary
model_efficientnetb1.summary()

Expected output:
```
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_1 (InputLayer)         [(None, 240, 120, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, None, None, 3)     0         
_________________________________________________________________
efficientnetb1 (Functional)  (None, 8, 4, 1280)        6575239   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
=================================================================
Total params: 6,577,801
Trainable params: 6,515,746
Non-trainable params: 62,055
_________________________________________________________________
```

In [ ]:
# setup list to save the training history
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
HISTORY_HISTORY_ACCURACY = []
HISTORY_HISTORY_VALACCURACY = []
HISTORY_HISTORY_LOSS = []
HISTORY_HISTORY_VALLOSS = []

### Model training (5-fold cross validation)

For each fold, the training and validation set split is in term of patient (similar to the train and test split) to avoid data leak.

In [ ]:
# starting from fold-1
fold_var = 1

# 5-fold cross validation
for train_index, val_index in gkf.split(train_val_df, groups=train_val_df['PatientName']):
    print("Fold - ",fold_var)
    # set name for the model and weight (.h5) for this fold.
    this_model_path = set_model_path+"model_fold"+str(fold_var)+".h5"
    
    # allocate train and val set according to the split indix
    train_df = train_val_df.iloc[train_index] # training set
    val_df = train_val_df.iloc[val_index] # validation set

    # get the file (X: in file path as type of string) and the label (y).
    X_train = train_df.filepath
    y_train = train_df.abnormal
    X_val = val_df.filepath
    y_val = val_df.abnormal

    # find class weights for training set in this fold
    class_weight = find_binary_class_weights(y_train)

    # build the dataset from the file paths and the labels
    training_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    validation_data = tf.data.Dataset.from_tensor_slices((X_val, y_val))
    training_data = training_data.map(load_image_and_label_from_path, num_parallel_calls=AUTOTUNE)
    validation_data = validation_data.map(load_image_and_label_from_path, num_parallel_calls=AUTOTUNE)
    training_data = training_data.map(augment_batch) # augmentation

    # create the batches
    training_data_batches = training_data.shuffle(buffer_size=1000).batch(batch_size).prefetch(buffer_size=AUTOTUNE)
    validation_data_batches = validation_data.shuffle(buffer_size=1000).batch(batch_size).prefetch(buffer_size=AUTOTUNE)

    # import the model
    model = model_efficientnetb1
    # set the optimizer and callback
    decay_steps = int(round(len(X_train)/batch_size))*epochs
    cosine_decay = CosineDecay(initial_learning_rate=initial_learning_rate, decay_steps=decay_steps, alpha=alpha)
    callbacks = [ModelCheckpoint(filepath=this_model_path, monitor='val_loss', save_best_only=True)]
    model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(cosine_decay), metrics=["accuracy"])

    # train the model
    history = model.fit(training_data_batches,
                        epochs = epochs, 
                        validation_data=validation_data_batches,
                        class_weight=class_weight,
                        callbacks=callbacks)

    # save the whole model (structure and weight)
    model.save(this_model_path)

    # update the model training history
    HISTORY_HISTORY_ACCURACY.append(history.history['accuracy'])
    HISTORY_HISTORY_VALACCURACY.append(history.history['val_accuracy'])
    HISTORY_HISTORY_LOSS.append(history.history['loss'])
    HISTORY_HISTORY_VALLOSS.append(history.history['val_loss'])

    # load model to evaluate the performance of the model on the validation set in this fold.
    model = load_model(this_model_path)

    print("Evaluate")
    results = model.evaluate(validation_data_batches)
    results = dict(zip(model.metrics_names,results))

    # update the model validation history
    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDAITON_LOSS.append(results['loss'])

    # print the validation performance
    print("Validation accuracy = ",results['accuracy'])
    print("Validation loss = ",results['loss'])
    
    # go to next fold
    fold_var += 1
    print("---------------------------------------------------------------")

In [ ]:
# print the training and validation history for record.
print("HISTORY_HISTORY_ACCURACY = ",HISTORY_HISTORY_ACCURACY)
print("HISTORY_HISTORY_VALACCURACY = ",HISTORY_HISTORY_VALACCURACY)
print("HISTORY_HISTORY_LOSS = ",HISTORY_HISTORY_LOSS)
print("HISTORY_HISTORY_VALLOSS = ",HISTORY_HISTORY_VALLOSS)
print("VALIDATION_ACCURACY = ",VALIDATION_ACCURACY)
print("VALIDAITON_LOSS = ",VALIDAITON_LOSS)

In [28]:
# locate the 5 models in the 5-fold cross validation model
model_fold1_path = set_model_path + 'model_fold1.h5'
model_fold2_path = set_model_path + 'model_fold2.h5'
model_fold3_path = set_model_path + 'model_fold3.h5'
model_fold4_path = set_model_path + 'model_fold4.h5'
model_fold5_path = set_model_path + 'model_fold5.h5'
model_path_list = [model_fold1_path,model_fold2_path,model_fold3_path,model_fold4_path,model_fold5_path]

### Making and recording the predictions on the train-validation set

In [ ]:
# build the training-validation set (without shuffling)
X_trainval = train_val_df.filepath
y_trainval = train_val_df.abnormal
trainval_data = tf.data.Dataset.from_tensor_slices((X_trainval, y_trainval))
trainval_data = trainval_data.map(load_image_and_label_from_path, num_parallel_calls=AUTOTUNE)
trainval_data_batches = trainval_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

In [ ]:
# recording the model predictions on the training-validation set for each fold model
trainval_result_df = train_val_df
for current_model_path in model_path_list:
    model_current = load_model(current_model_path)
    # Classification on the train-val set
    test_predictions = model_current.predict(trainval_data_batches,verbose=1)
    # Get most likely class
    predicted_classes = np.argmax(test_predictions, axis=1)
    current_model_basename = os.path.basename(current_model_path)
    current_model_name = os.path.splitext(current_model_basename)[0]
    trainval_result_df["test_prediction_"+current_model_name] = test_predictions[:,1]
    trainval_result_df["predicted_classes_"+current_model_name] = predicted_classes

In [ ]:
# save the 5 models predictions on the training-validation set from the 5-fold model
trainval_result_df.to_csv("trainval_5fold_df.csv",index=False)

### Making and recording predictions on the independent test set

In [22]:
# build the test set
X_test = test_df.filepath
y_test = test_df.abnormal
test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_data = test_data.map(load_image_and_label_from_path, num_parallel_calls=AUTOTUNE)
test_data_batches = test_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

In [ ]:
# recording the model predictions on the test set for each fold model
test_result_df = test_df
for current_model_path in model_path_list:
    model_current = load_model(current_model_path)
    # make predictions on the test set
    test_predictions = model_current.predict(test_data_batches,verbose=1)
    # Get most likely class
    predicted_classes = np.argmax(test_predictions, axis=1)
    current_model_basename = os.path.basename(current_model_path)
    current_model_name = os.path.splitext(current_model_basename)[0]
    test_result_df["test_prediction_"+current_model_name] = test_predictions[:,1]
    test_result_df["predicted_classes_"+current_model_name] = predicted_classes

In [33]:
# save the 5 models predictions on the independent test set from the 5-fold model
test_result_df.to_csv("test_5fold_df.csv",index=False)